In [1]:
from __future__ import print_function, unicode_literals, division
from io import open
import glob

In [2]:
def findFiles(path):
    return glob.glob(path)
print (findFiles('data/names/*.txt'))

[u'data/names/Polish.txt', u'data/names/Arabic.txt', u'data/names/Chinese.txt', u'data/names/German.txt', u'data/names/Dutch.txt', u'data/names/Irish.txt', u'data/names/French.txt', u'data/names/Russian.txt', u'data/names/Vietnamese.txt', u'data/names/Japanese.txt', u'data/names/Korean.txt', u'data/names/Italian.txt', u'data/names/English.txt', u'data/names/Scottish.txt', u'data/names/Greek.txt', u'data/names/Spanish.txt', u'data/names/Portuguese.txt', u'data/names/Czech.txt']


In [3]:
import unicodedata
import string

In [4]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
print (n_letters)
def unicodetoAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicodetoAscii('Ślusàrski'))

57
Slusarski


In [5]:
category_lines = {}
all_categories = []
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodetoAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
n_categories = len(all_categories)

In [6]:
print (n_categories)

18


In [7]:
print (category_lines['Italian'][:5])

[u'Abandonato', u'Abatangelo', u'Abatantuono', u'Abate', u'Abategiovanni']


In [8]:
import torch
from torch.autograd import Variable

In [9]:
def letterToIndex(letter):
    return all_letters.find(letter)
def linetoTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


In [10]:
print (linetoTensor('Jones').size())

torch.Size([5, 1, 57])


In [11]:
a = [[1, 2, 3]]
a_tensor = torch.Tensor(a)
print (a_tensor.size())
b = [1, 2, 3]
b_tensor = torch.Tensor(b)
print (b_tensor.size())

torch.Size([1, 3])
torch.Size([3])


In [12]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2o = torch.nn.Linear(input_size + hidden_size, output_size)
        self.i2h = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.softmax = torch.nn.LogSoftmax(dim=1)
        
    def forward(self, inpt, hidden):
        
        combined = torch.cat((inpt, hidden), 1)
        next_hidden = self.i2h(combined)
        out = self.i2o(combined)
        output = self.softmax(out)
        
        return output, next_hidden
    
    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))
        

In [13]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

inpt = Variable(linetoTensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))
output, next_hidden = rnn(inpt[0], hidden)
print(output.size())
print(output)

torch.Size([1, 18])
Variable containing:

Columns 0 to 9 
-2.9990 -2.8997 -2.9557 -2.9799 -2.8219 -2.8266 -2.9045 -2.9325 -2.8477 -2.8816

Columns 10 to 17 
-2.9103 -2.9486 -2.9667 -2.9046 -2.7912 -2.7471 -2.7880 -2.9683
[torch.FloatTensor of size 1x18]



In [14]:
def categoryfromOutput(output):
    top_n, top_i = output.data.topk(1)
    category_i =top_i[0][0]
    return all_categories[category_i], category_i
categoryfromOutput(output)

(u'Spanish', 15)

In [15]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l)-1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(linetoTensor(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print ('category =', category, '/ line =', line)
    

category = German / line = Wieck
category = Korean / line = Sin
category = Scottish / line = Bruce
category = Greek / line = Katsourinis
category = Irish / line = Mooney
category = Dutch / line = Lucas
category = Japanese / line = Joshuya
category = Scottish / line = Fraser
category = Chinese / line = Tong
category = Scottish / line = Sutherland


In [16]:
criterion = torch.nn.NLLLoss()

In [17]:
learning_rate = 0.005

def train(category_tensor, line_tensor):
    
    hidden = rnn.initHidden()
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    loss = criterion(output, category_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.data[0]

In [18]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    if iter % print_every == 0:
        guess, guess_i = categoryfromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))


    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 7s) 2.9146 Gaspar / German ✗ (Portuguese)
10000 10% (0m 13s) 1.1494 Toyama / Japanese ✓
15000 15% (0m 21s) 1.3863 Niu / Korean ✗ (Chinese)
20000 20% (0m 29s) 3.5606 Leon / Korean ✗ (French)
25000 25% (0m 36s) 1.5480 Kabaidze / Russian ✓
30000 30% (0m 44s) 1.6826 Aritza / Portuguese ✗ (Spanish)
35000 35% (0m 52s) 2.4304 Mata / Japanese ✗ (Portuguese)
40000 40% (1m 0s) 3.4015 Konoe / English ✗ (Japanese)
45000 45% (1m 8s) 0.8322 Montanari / Italian ✓
50000 50% (1m 15s) 3.8950 Kan / Chinese ✗ (Dutch)
55000 55% (1m 23s) 1.4941 Diarmaid / French ✗ (Irish)
60000 60% (1m 31s) 0.7657 Ryu / Korean ✓
65000 65% (1m 38s) 2.1657 Shichirobei / Polish ✗ (Japanese)
70000 70% (1m 45s) 1.1683 Stangl / Czech ✓
75000 75% (1m 52s) 1.7455 Reid / French ✗ (Scottish)
80000 80% (2m 0s) 0.7045 Quigley / Irish ✓
85000 85% (2m 8s) 1.1144 White / Scottish ✓
90000 90% (2m 15s) 0.0387 Czajkowski / Polish ✓
95000 95% (2m 23s) 0.3478 Ribeiro / Portuguese ✓
100000 100% (2m 31s) 0.0169 Antonakos / Greek ✓


In [19]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

In [20]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    output = evaluate(Variable(linetoTensor(input_line)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        category_index = topi[0][i]
        print('(%.2f) %s' % (value, all_categories[category_index]))
        predictions.append([value, all_categories[category_index]])

predict('Howard')
predict('Chao')
predict('Nakamoto')


> Howard
(-0.99) English
(-1.90) Polish
(-2.07) Scottish

> Chao
(-1.23) Vietnamese
(-1.71) Chinese
(-2.14) Korean

> Nakamoto
(-0.29) Japanese
(-2.18) Russian
(-3.13) German
